In [2]:
###### 1.0 Install LIWC'S EQUIVALENT ######

'''
liwc pkg could be downloaded but it requires lexicon to use, which has to be purchsed.
see: https://pypi.org/project/liwc/
!pip install -U liwc

alternatively we could use EMPATH
Guideline here: https://github.com/Ejhfast/empath-client

If still not working: 
Try out SEANCE. It's free, requires no programming experience, outperforms LIWC, and is open science
https://www.linguisticanalysistools.org/seance.html
'''

!pip install empath spacy
from empath import Empath
lexicon = Empath()

In [3]:
###### EXPLORING EMPATHY ######

# Print all category (class) names
print(list(lexicon.cats.keys()))

['help', 'office', 'dance', 'money', 'wedding', 'domestic_work', 'sleep', 'medical_emergency', 'cold', 'hate', 'cheerfulness', 'aggression', 'occupation', 'envy', 'anticipation', 'family', 'vacation', 'crime', 'attractive', 'masculine', 'prison', 'health', 'pride', 'dispute', 'nervousness', 'government', 'weakness', 'horror', 'swearing_terms', 'leisure', 'suffering', 'royalty', 'wealthy', 'tourism', 'furniture', 'school', 'magic', 'beach', 'journalism', 'morning', 'banking', 'social_media', 'exercise', 'night', 'kill', 'blue_collar_job', 'art', 'ridicule', 'play', 'computer', 'college', 'optimism', 'stealing', 'real_estate', 'home', 'divine', 'sexual', 'fear', 'irritability', 'superhero', 'business', 'driving', 'pet', 'childish', 'cooking', 'exasperation', 'religion', 'hipster', 'internet', 'surprise', 'reading', 'worship', 'leader', 'independence', 'movement', 'body', 'noise', 'eating', 'medieval', 'zest', 'confusion', 'water', 'sports', 'death', 'healing', 'legend', 'heroic', 'celebr

In [48]:
###### WHAT EXISTING CLASSES IN EMPATHY COULD BE ADOPTED ######

# Use Yarkoni (2011) Table 1 as the benchmark

# Define the class (category) we're looking for
class_name = "SEE"  
# Find matching categories (sub-string match, case insensitive)
matched_categories = [cat for cat in lexicon.cats if class_name.lower() in cat.lower()]
# Check if any categories matched and print the results
if matched_categories:
    print("YES")
    print("Matched categories:", matched_categories)
else:
    print("NO")

'''
EMPATHY: 
affect, positive_emotions, negative_emotions, anger, sadness, hearing, communication, friends, family, swearing_terms

SPACY:
pronouns(PRON), articles(DET), prepositions(PREP), numbers(NUM)

Maybe based on SPACY:
1st person sg/pl, 2nd person, 3rd person pronouns
Past/present/future tense vb.
'''

NO


'\n'

In [1]:
# do an example analysis
result = lexicon.analyze("he kiss the other person", normalize=True)
filtered_result = {category: value for category, value in result.items() if value > 0}
print(filtered_result)

NameError: name 'lexicon' is not defined

In [49]:
### PERSONAL PRONOUNS ###

import spacy

# 加载英语模型
nlp = spacy.load("en_core_web_sm")

# 定义一个函数来识别人称代词
def identify_personal_pronouns(text):
    doc = nlp(text)
    pronouns = {
        "1st person singular": [],
        "1st person plural": [],
        "2nd person": [],
        "3rd person singular": [],
        "3rd person plural": []
    }
    
    for token in doc:
        if token.pos_ == "PRON":  # 仅检查代词
            if token.text.lower() in ["i", "me", "my", "mine"]:  # 第一人称单数
                pronouns["1st person singular"].append(token.text)
            elif token.text.lower() in ["we", "us", "our", "ours"]:  # 第一人称复数
                pronouns["1st person plural"].append(token.text)
            elif token.text.lower() in ["you", "your", "yours"]:  # 第二人称
                pronouns["2nd person"].append(token.text)
            elif token.text.lower() in ["he", "him", "his", "she", "her", "hers", "it", "its"]:  # 第三人称单数
                pronouns["3rd person singular"].append(token.text)
            elif token.text.lower() in ["they", "them", "their", "theirs"]:  # 第三人称复数
                pronouns["3rd person plural"].append(token.text)
    
    return pronouns

# 示例句子
text = "I have a friend, and he said that we should meet you and them."
result = identify_personal_pronouns(text)
print(result)


{'1st person singular': ['I'], '1st person plural': ['we'], '2nd person': ['you'], '3rd person singular': ['he'], '3rd person plural': ['them']}


In [50]:
### VERB TENSE ###

# 定义一个函数来识别动词时态
def identify_verb_tenses(text):
    doc = nlp(text)
    tenses = {
        "past": [],
        "present": [],
        "future": []
    }
    
    for token in doc:
        if token.pos_ == "VERB":  # 仅检查动词
            # 检查时态
            if token.tag_ in ["VBD", "VBN"]:  # 过去时或过去分词
                tenses["past"].append(token.text)
            elif token.tag_ in ["VBZ", "VBP", "VB"]:  # 现在时
                tenses["present"].append(token.text)
            elif token.text.lower().startswith("will") or token.tag_ == "MD":  # 将来时
                tenses["future"].append(token.text)
    
    return tenses

# 示例句子
text = "I will go to the store tomorrow. He went there yesterday, and I am here now."
result = identify_verb_tenses(text)
print(result)

{'past': ['went'], 'present': ['go'], 'future': []}


In [51]:
### NEGATIONS ###
# 定义一个函数来识别否定词
def identify_negations(text):
    doc = nlp(text)
    negations = []
    
    for token in doc:
        if token.dep_ == "neg":  # 检查是否为否定词
            negations.append(token.text)
    
    return negations

# 示例句子
text = "I do not like apples, but I love oranges. He isn't coming to the party."
result = identify_negations(text)
print(result)

['not', "n't"]


In [ ]:
###  Data processing ###

# Database: lines per person
# Put the sentences in, give each a lexicon marker, this will be a new category
# Use previous studies' conclusion on characters' BIG-5, 
#   we explore if correlations between Big Five personality traits and LIWC categories
#   align with conclusions from Yarkoni (2011) Table 1

In [ ]:
'''
Word / lexicon / character / Freq.

character personality tendency

coefficient
'''